### Importing Libraries

In [1]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd

import requests_cache #requests_cache to cache API call data
import time
from IPython.core.display import clear_output
from tqdm import tqdm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# keeping data from API calls into cache
requests_cache.install_cache()

### API Calls

While Loop over api.spacexdata (great API !) - over launch pages

Append list with API call results, to then make a DataFrame out of it
All calls cached locally with requests_cache : limit amount of calls needed and ease on the API server

In [7]:
page = 1
final_page = 100 #final page for while loop

responses_list = []

#loop over page with url launches/page
while page < final_page:
    print('The rocket is on the launch pad - API call ignition')
    print(f'SpaceX API call - requesting page {page}/{page}')
    
    # clear the output of each print to make things neater
    clear_output(wait = True)
    
    payload = {'limit':500,
    'page':page}
    url = f'https://api.spacexdata.com/v3/launches/{page}'
    
    response = requests.get(url)
    
    #back stop if error = stop looping
    if response.status_code != 200:
        print('limit page result reached on page number',page,response.text, '- end of API call')
        break
    
    #if it's not a cached result, sleep
    if not getattr(response, 'from_cache', False):
        time.sleep(0.35)
    
    #append response list
    responses_list.append(response)
    
    #increment page number
    page += 1
    
    print("Requested page {}/{}".format(page, page), 'SpaceX API call completed')

Requested page 100/100 SpaceX API call completed


###### testing API call on 1st launch

In [20]:
url = 'https://api.spacexdata.com/v3/launches/'

result = requests.get(f'https://api.spacexdata.com/v3/launches/1')

print(json.dumps(result.json(), indent=4))

{
    "flight_number": 1,
    "mission_name": "FalconSat",
    "mission_id": [],
    "upcoming": false,
    "launch_year": "2006",
    "launch_date_unix": 1143239400,
    "launch_date_utc": "2006-03-24T22:30:00.000Z",
    "launch_date_local": "2006-03-25T10:30:00+12:00",
    "is_tentative": false,
    "tentative_max_precision": "hour",
    "tbd": false,
    "launch_window": 0,
    "rocket": {
        "rocket_id": "falcon1",
        "rocket_name": "Falcon 1",
        "rocket_type": "Merlin A",
        "first_stage": {
            "cores": [
                {
                    "core_serial": "Merlin1A",
                    "flight": 1,
                    "block": null,
                    "gridfins": false,
                    "legs": false,
                    "reused": false,
                    "land_success": null,
                    "landing_intent": false,
                    "landing_type": null,
                    "landing_vehicle": null
                }
            ]
     

dict of dictionaries

In [10]:
#dummy df of JSON call & normalize JSON to picture final df result
r0 = responses_list[0]
r0_json = json_normalize(r0.json())
df1 = pd.DataFrame(r0_json)
df1.head()

flight_number mission_name mission_id  upcoming launch_year  \
0              1    FalconSat         []     False        2006   

   launch_date_unix           launch_date_utc          launch_date_local  \
0        1143239400  2006-03-24T22:30:00.000Z  2006-03-25T10:30:00+12:00   

   is_tentative tentative_max_precision    tbd  launch_window ships  \
0         False                    hour  False              0    []   

   launch_success                                           details  \
0           False  Engine failure at 33 seconds and loss of vehicle   

       static_fire_date_utc  static_fire_date_unix  crew rocket.rocket_id  \
0  2006-03-17T00:00:00.000Z             1142553600  None          falcon1   

  rocket.rocket_name rocket.rocket_type  \
0           Falcon 1           Merlin A   

                            rocket.first_stage.cores  \
0  [{'core_serial': 'Merlin1A', 'flight': 1, 'blo...   

   rocket.second_stage.block  \
0                          1   

                        rocket.second_stage.payloads  rocket.fairings.reused  \
0  [{'payload_id': 'FalconSAT-2', 'norad_id': [],...                   False   

   rocket.fairings.recovery_attempt  rocket.fairings.recovered  \
0                             False                      False   

  rocket.fairings.ship telemetry.flight_club launch_site.site_id  \
0                 None                  None     kwajalein_atoll   

  launch_site.site_name     launch_site.site_name_long  \
0       Kwajalein Atoll  Kwajalein Atoll Omelek Island   

   launch_failure_details.time launch_failure_details.altitude  \
0                           33                            None   

  launch_failure_details.reason  \
0         merlin engine failure   

                               links.mission_patch  \
0  https://images2.imgbox.com/40/e3/GypSkayF_o.png   

                         links.mission_patch_small links.reddit_campaign  \
0  https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png                  None   

  links.reddit_launch links.reddit_recovery links.reddit_media links.presskit  \
0                None                  None               None           None   

                                  links.article_link  \
0  https://www.space.com/2196-spacex-inaugural-fa...   

                         links.wikipedia  \
0  https://en.wikipedia.org/wiki/DemoSat   

                              links.video_link links.youtube_id  \
0  https://www.youtube.com/watch?v=0a_00nJ_Y88      0a_00nJ_Y88   

  links.flickr_images  timeline.webcast_liftoff  
0                  []                        54

In [53]:
df1.columns

Index(['flight_number', 'mission_name', 'mission_id', 'upcoming',
       'launch_year', 'launch_date_unix', 'launch_date_utc',
       'launch_date_local', 'is_tentative', 'tentative_max_precision', 'tbd',
       'launch_window', 'ships', 'launch_success', 'details',
       'static_fire_date_utc', 'static_fire_date_unix', 'crew',
       'rocket.rocket_id', 'rocket.rocket_name', 'rocket.rocket_type',
       'rocket.first_stage.cores', 'rocket.second_stage.block',
       'rocket.second_stage.payloads', 'rocket.fairings.reused',
       'rocket.fairings.recovery_attempt', 'rocket.fairings.recovered',
       'rocket.fairings.ship', 'telemetry.flight_club', 'launch_site.site_id',
       'launch_site.site_name', 'launch_site.site_name_long',
       'launch_failure_details.time', 'launch_failure_details.altitude',
       'launch_failure_details.reason', 'links.mission_patch',
       'links.mission_patch_small', 'links.reddit_campaign',
       'links.reddit_launch', 'links.reddit_recovery', 'lin

### Listing variables

**Important variables to keep**

- 'launch_year' : deprecated 
- 'launch_date_unix' : 
- 'launch_date_utc', :
- - 'crew', : 
- 'ships' : 
- 'launch_success' : True when success
- 'details', : 
- 'launch_site.site_id', : 
- 'launch_site.site_name' : 
- 'launch_site.site_name_long', : 
- 'launch_failure_details.time' : 
- 'launch_failure_details.altitude', : 
- 'launch_failure_details.reason' : 

**ID related**
- flight_number : 
- mission_name : 
- mission_id : 

**Launch related**
- launch_year : deprecated 
- launch_date_unix : 
- launch_date_utc :
- - crew : 
- ships : 
- launch_success : True when success
- details : 
- launch_site.site_id : 
- launch_site.site_name : 
- launch_site.site_name_long : 
- launch_failure_details.time : 
- launch_failure_details.altitude : 
- launch_failure_details.reason : 

**Rocket related**
- rocket.rocket_id : 
- rocket.rocket_name : 
- rocket.rocket_type :
- rocket.second_stage.payloads : 
- rocket.fairings.reused : 
- rocket.fairings.recovery_attempt : 
- rocket.fairings.recovered : 

**Useless**
- ditching all links columns = only article & PNG links
- upcoming' : Set as true until the moment of launch

### for loop to create final df with all calls from appended list

In [55]:
#empty list to append all API calls
frame = [] 
for i in range(len(responses_list)):
    r = responses_list[i]
    df_temp = pd.DataFrame(json_normalize(r.json()))
    frame.append(df_temp)
df_launch_raw = pd.concat(frame, sort=False)

In [56]:
print(len(responses_list))

type(responses_list[0])

99


requests.models.Response

In [57]:
df_launch_raw

flight_number               mission_name mission_id  upcoming launch_year  \
0               1                  FalconSat         []     False        2006   
0               2                    DemoSat         []     False        2007   
0               3                Trailblazer         []     False        2008   
0               4                     RatSat         []     False        2008   
0               5                   RazakSat         []     False        2009   
..            ...                        ...        ...       ...         ...   
0              95                 Starlink 7         []      True        2020   
0              96  Starlink-8 & SkySat 16-18         []      True        2020   
0              97                 Starlink-9         []      True        2020   
0              98    GPS III SV03 (Columbus)         []      True        2020   
0              99                      SXM-7         []      True        2020   

    launch_date_unix           launch_date_utc          launch_date_local  \
0         1143239400  2006-03-24T22:30:00.000Z  2006-03-25T10:30:00+12:00   
0         1174439400  2007-03-21T01:10:00.000Z  2007-03-21T13:10:00+12:00   
0         1217648040  2008-08-02T03:34:00.000Z  2008-08-02T15:34:00+12:00   
0         1222643700  2008-09-28T23:15:00.000Z  2008-09-28T11:15:00+12:00   
0         1247456100  2009-07-13T03:35:00.000Z  2009-07-13T15:35:00+12:00   
..               ...                       ...                        ...   
0         1591056000  2020-06-02T00:00:00.000Z  2020-06-01T20:00:00-04:00   
0         1590969600  2020-06-01T00:00:00.000Z  2020-05-31T20:00:00-04:00   
0         1590969600  2020-06-01T00:00:00.000Z  2020-05-31T20:00:00-04:00   
0         1593475200  2020-06-30T00:00:00.000Z  2020-06-29T20:00:00-04:00   
0         1596326400  2020-08-02T00:00:00.000Z  2020-08-01T20:00:00-04:00   

    is_tentative tentative_max_precision    tbd launch_window ships  \
0          False                    hour  False             0    []   
0          False                    hour  False             0    []   
0          False                    hour  False             0    []   
0          False                    hour  False             0    []   
0          False                    hour  False             0    []   
..           ...                     ...    ...           ...   ...   
0           True                     day  False          None    []   
0           True                   month  False          None    []   
0           True                   month  False          None    []   
0           True                     day  False          None    []   
0           True                     day  False          None    []   

   launch_success                                            details  \
0           False   Engine failure at 33 seconds and loss of vehicle   
0           False  Successful first stage burn and transition to ...   
0           False  Residual stage 1 thrust led to collision betwe...   
0            True  Ratsat was carried to orbit on the first succe...   
0            True                                               None   
..            ...                                                ...   
0            None                                               None   
0            None                                               None   
0            None                                               None   
0            None                                               None   
0            None                                               None   

        static_fire_date_utc static_fire_date_unix  crew rocket.rocket_id  \
0   2006-03-17T00:00:00.000Z            1142553600  None          falcon1   
0                       None                  None  None          falcon1   
0                       None                  None  None          falcon1   
0   2008-09-20T00:00:00.000Z            1221868800  None          falcon1   
0  

In [58]:
df_launch_raw.to_csv('launch_raw.csv', index=False)

### Series of columns to keep

In [ ]:
## 

In [46]:
Rocket_related = pd.Series(
['rocket_id',
'rocket_name',
'rocket_type',
'second_stage.payloads',
'fairings.reused',
'fairings.recovery_attempt',
'fairings.recovered'])

In [47]:
rocket = pd.DataFrame(Rocket_related)
rocket

0
0                  rocket_id
1                rocket_name
2                rocket_type
3      second_stage.payloads
4            fairings.reused
5  fairings.recovery_attempt
6         fairings.recovered

In [49]:
Launch_related = pd.Series(
['launch_year', 
'launch_date_unix', 
'launch_date_utc',
'crew', 
'ships', 
'launch_success',
'details', 
'launch_site.site_id', 
'launch_site.site_name', 
'launch_site.site_name_long', 
'launch_failure_details.time', 
'launch_failure_details.altitude', 
'launch_failure_details.reason'])

launch = pd.DataFrame(Launch_related)
launch

0
0                       launch_year
1                  launch_date_unix
2                   launch_date_utc
3                              crew
4                             ships
5                    launch_success
6                           details
7               launch_site.site_id
8             launch_site.site_name
9        launch_site.site_name_long
10      launch_failure_details.time
11  launch_failure_details.altitude
12    launch_failure_details.reason